In [2]:
# import library
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [3]:
from bs4 import BeautifulSoup

with open("/Users/alifmahardhika/Desktop/ANAMEDSOS/crawlwlwl/mu-pl-results.html") as fp:
    soup = BeautifulSoup(fp, 'html.parser')

# soup.find( class_ = "fixtures__matches-list" )
fixture_url_list = soup.find_all('div', class_='fixture postMatch')
urls = []
for i in fixture_url_list:
    urls.append("https://{}".format(i['data-href'][2:]))

In [4]:
def get_home_or_away(driver):
    team_classes = driver.find_elements_by_class_name("home")
    for i in team_classes:
        j = i.find_elements_by_class_name("teamName") 
        if(len(j) != 0):
            if(j[0].find_element_by_class_name("short").get_attribute('innerHTML') == 'MUN'):
                return 'home'
            else: return 'away'
def get_scorer_and_assist(text_to_parse):
    a = text_to_parse.split(".")
    goal_scorer = a[1].split(" (")[0][1:]
    # goal_scorer = goal_scorer[0]
    assist = a[2].split(" ")[3]
    # return in format "scorer,assister"
    return "{},{}".format(goal_scorer,assist)

def scrape(url):
    driver = webdriver.Chrome("/Users/alifmahardhika/Desktop/ANAMEDSOS/chromedriver")
    driver.get(url)
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located(
            (By.XPATH, '//div[@class="blogCard goal"]/*')))
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        goals = driver.find_elements_by_xpath('//div[@class="blogCard goal"]/*')

        global res
        for i in goals:  
            a = i.get_attribute('innerHTML')
            if 'Assisted by' in a and a.count("Manchester United") > 1:
                res.append(get_scorer_and_assist(i.find_element_by_tag_name("p").get_attribute('innerHTML')))
    except Exception as e:
        # print(e)
        pass
    print('done')
    driver.quit()

In [5]:
res = []
count = 0
for url in urls:
    scrape(url)
    # count +=1
    # if count== 5:
    #     break
print(res)

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
['Edinson Cavani,Donny', 'Mason Greenwood,Paul', 'Mason Greenwood,Paul', 'Edinson Cavani,Mason', 'Mason Greenwood,Paul', 'Daniel James,Bruno', 'Anthony Martial,Bruno', 'Harry Maguire,Alex', 'Marcus Rashford,Bruno', 'Bruno Fernandes,Edinson', 'Marcus Rashford,Juan', 'Mason Greenwood,Alex', 'Edinson Cavani,Marcus', 'Edinson Cavani,Bruno', 'Marcus Rashford,Bruno', 'Aaron Wan-Bissaka,Marcus', 'Bruno Fernandes,Marcus']


In [39]:
# print(len(res))
assist_goal = []
names = []
for i in res:
    i = i.split(",")
    goal = i[0].split(" ")[0]
    assist = i[1]
    assist_goal.append('{},{}'.format(assist, goal))
    names.append(goal)
    names.append(assist)

print("assist relation:")
print(assist_goal)
names = sorted(list(dict.fromkeys(names)))
names.insert(0, "players")
print("\nplayer list:")
print(names)


assist relation:
['Donny,Edinson', 'Paul,Mason', 'Paul,Mason', 'Mason,Edinson', 'Paul,Mason', 'Bruno,Daniel', 'Bruno,Anthony', 'Alex,Harry', 'Bruno,Marcus', 'Edinson,Bruno', 'Juan,Marcus', 'Alex,Mason', 'Marcus,Edinson', 'Bruno,Edinson', 'Bruno,Marcus', 'Marcus,Aaron', 'Marcus,Bruno']

player list:
['players', 'Aaron', 'Alex', 'Anthony', 'Bruno', 'Daniel', 'Donny', 'Edinson', 'Harry', 'Juan', 'Marcus', 'Mason', 'Paul']


In [42]:
import csv

empty_row = []
for i in range(len(names) -1):
    empty_row.append(0)
with open('man_utd_assist_network_pl20-21.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(names) #first row
    for i in names:
        if i == "players": continue
        row = [i] + empty_row
        writer.writerow(row) #first row

In [62]:
import pandas as pd
df = pd.read_csv("man_utd_assist_network_pl20-21.csv")
print(df.head(5))

   players  Aaron  Alex  Anthony  Bruno  Daniel  Donny  Edinson  Harry  Juan  \
0    Aaron      0     0        0      0       0      0        0      0     0   
1     Alex      0     0        0      0       0      0        0      0     0   
2  Anthony      0     0        0      0       0      0        0      0     0   
3    Bruno      0     0        0      0       0      0        0      0     0   
4   Daniel      0     0        0      0       0      0        0      0     0   

   Marcus  Mason  Paul  
0       0      0     0  
1       0      0     0  
2       0      0     0  
3       0      0     0  
4       0      0     0  


In [100]:
names.index('Aaron')

1

In [106]:
df_copy = df.copy()
for i in assist_goal:
    assist = i.split(',')[0]
    goal = i.split(',')[1]
    df_copy.at[names.index(assist)-1,goal] = df_copy.at[names.index(assist)-1,goal] +1
print(df_copy.head(6))


   players  Aaron  Alex  Anthony  Bruno  Daniel  Donny  Edinson  Harry  Juan  \
0    Aaron      0     0        0      0       0      0        0      0     0   
1     Alex      0     0        0      0       0      0        0      1     0   
2  Anthony      0     0        0      0       0      0        0      0     0   
3    Bruno      0     0        1      0       1      0        1      0     0   
4   Daniel      0     0        0      0       0      0        0      0     0   
5    Donny      0     0        0      0       0      0        1      0     0   

   Marcus  Mason  Paul  
0       0      0     0  
1       0      1     0  
2       0      0     0  
3       2      0     0  
4       0      0     0  
5       0      0     0  


In [107]:
df_copy.to_csv('out.csv', index=False)  